In [1]:
#Final Project
#Each layer had to be prepared with geoprocessing tools before it could be used

In [2]:
#inport the necessary packages and set up the environment
import arcpy
from arcpy import env
from arcpy.sa import *

# Overwrite
env.overwriteOutput = True

# set up workspace
env.workspace = "C:\\Users\\arigr\\Documents\\geog380\\final_project"

In [3]:
#Step 1: prepare data
#1a: Prepare population data
inputshp = "caltracts/tl_2016_06_tract.shp"
inputcsv = "caltractppop.csv"
outputshp = "population.shp"
outputtif = "population.tif"

#Join the census tract shapefile data with the census tract population data
#Convert to raster
arcpy.MakeFeatureLayer_management(inputshp, "tempLayer")
arcpy.AddJoin_management("tempLayer", "GEOIDD", inputcsv, "GEOID", "KEEP_COMMON")
arcpy.FeatureToRaster_conversion("tempLayer", "POP100", outputtif)
print(arcpy.GetMessages())


Start Time: Tuesday, December 15, 2020 7:45:40 PM
Succeeded at Tuesday, December 15, 2020 7:45:44 PM (Elapsed Time: 4.44 seconds)


In [4]:
#1b: Prepare housing unit data
inputshp = "caltracts/tl_2016_06_tract.shp"
inputcsv = "caltractppop.csv"
outputshp = "housing.shp"
outputtif = "housing.tif"

#Join the census tract shapefile data with the census tract population data
#Convert to raster
arcpy.MakeFeatureLayer_management(inputshp, "tempLayer")
arcpy.AddJoin_management("tempLayer", "GEOIDD", inputcsv, "GEOID", "KEEP_COMMON")
arcpy.FeatureToRaster_conversion("tempLayer", "HU100", outputtif, 0.001)
print(arcpy.GetMessages())

Start Time: Tuesday, December 15, 2020 7:45:44 PM
Succeeded at Tuesday, December 15, 2020 7:45:59 PM (Elapsed Time: 14.70 seconds)


In [5]:
#1c: Prepare wind data
inputshp = "californiawindhighresolution/ca_50mwind.shp"
clipshp = "ca-state-boundary/CA_STATE_TIGER2016.shp"
tempshp = "tempwind.shp"
outputtif = "wind.tif"

#Clip the wind data to the California borders (it had ocean data as well)
#Convert to raster
arcpy.Clip_analysis(inputshp, clipshp, tempshp)
arcpy.MakeFeatureLayer_management(tempshp, "tempLayer")
arcpy.FeatureToRaster_conversion("tempLayer", "GRIDCODE", outputtif, 0.1)
print(arcpy.GetMessages())

Start Time: Tuesday, December 15, 2020 7:47:02 PM
Succeeded at Tuesday, December 15, 2020 7:47:16 PM (Elapsed Time: 13.80 seconds)


In [6]:
#1d: Prepare temperature data
#Some of this has already been done in ArcGIS Pro
#The temperature data was originally very complex
#I resampled the data to generalize the very specific points given

clipshp = "ca-state-boundary/CA_STATE_TIGER2016.shp"
inputtif = "October_16.tif"
outputtif = "temp.tif"

#Clip data to California borders (it was originally all of North America)
arcpy.Clip_management(inputtif,"", outputtif, clipshp, 0, "ClippingGeometry", "MAINTAIN_EXTENT")
print(arcpy.GetMessages())

Start Time: Tuesday, December 15, 2020 7:47:16 PM
Building Pyramids...
Succeeded at Tuesday, December 15, 2020 7:47:16 PM (Elapsed Time: 0.18 seconds)


In [7]:
#1e: Prepare power line data
inputshp = "California_Transmission_Lines/Transmission_Line.shp"
buffershp = "powerlinesbuffer.shp"
outputtif = "powerlines.tif"

#Add a 5 mile buffer around power lines (this was the distance from the Camp Fire ignition site)
#Add a new field call 'isPowerLin' ('isPowerLine' was too long for the program)
#If an area is within the buffer 'isPowerLin' = 1, otherwise 0
#Clip the raster to California boundaries

arcpy.Buffer_analysis(inputshp, buffershp, "5 Miles", "FULL", "ROUND")
arcpy.MakeFeatureLayer_management(buffershp, "tempLayer_")
arcpy.AddField_management("tempLayer_", "isPowerLin", "LONG", 8)
arcpy.CalculateField_management("tempLayer_", "isPowerLin", 1, "PYTHON3")
arcpy.FeatureToRaster_conversion("tempLayer_", "isPowerLin", "outputtifc")
clipshp = "ca-state-boundary/CA_STATE_TIGER2016.shp"
arcpy.MakeFeatureLayer_management(clipshp, "tempClipLayer")
arcpy.Clip_management("outputtifc","" , outputtif, clipshp, 0, "ClippingGeometry", "MAINTAIN_EXTENT")
print(arcpy.GetMessages())

Start Time: Tuesday, December 15, 2020 7:47:23 PM
Building Pyramids...
Succeeded at Tuesday, December 15, 2020 7:47:23 PM (Elapsed Time: 0.17 seconds)


In [8]:
#1f: Prepare fire risk data
#I had to convert gdb down to a single shapefile in arcGIS Pro
#Now all I have to do is resample and reclassify because the data is a little bit too complex
inputtif = "riskcomplex.tif"
needsreclasstif = "riskreclass.tif"
preclip = "riskpreclip.tif"
outputtif = "risk.tif"

#Resample the output because it was very complex
#Reclassify to set null values (which were represented by -128) to 0
arcpy.Resample_management(inputtif, needsreclasstif, "150 150", "NEAREST")
output = Reclassify(needsreclasstif, "Value", RemapValue([[-128, 0], [0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 5]]), "NODATA")
output.save(preclip)
#Clip raster to California boundaries
clipshp = "ca-state-boundary/CA_STATE_TIGER2016.shp"
arcpy.Clip_management(preclip,"", outputtif, clipshp, 0, "ClippingGeometry", "MAINTAIN_EXTENT")
print(arcpy.GetMessages())

Start Time: Tuesday, December 15, 2020 7:47:32 PM
Building Pyramids...
Succeeded at Tuesday, December 15, 2020 7:47:45 PM (Elapsed Time: 13.40 seconds)


In [9]:
#1g: Prepare fire frequency data
#just kidding this was already prepared
#I already converted PNG to TIF and added the georeferencing in arcGIS Pro
#It was a lot easier to add control points in arcGIS than it would have been in arcpy
#After this the raster was resampled (because of the border lines between regions)
#This removed the lines as much as possible
#Then the final output was reclassified so that places with lower than average fire cycles recieved value -1 and higher became 1
#All other became 0

#Clip raster to California boundaries
clipshp = "ca-state-boundary/CA_STATE_TIGER2016.shp"
inputtif = "pfridpreclip.tif"
outputtif = "pfrid.tif"
arcpy.Clip_management(inputtif,"", outputtif, clipshp, 0, "ClippingGeometry", "MAINTAIN_EXTENT")
print(arcpy.GetMessages())


Start Time: Tuesday, December 15, 2020 7:47:45 PM
Building Pyramids...
Succeeded at Tuesday, December 15, 2020 7:47:46 PM (Elapsed Time: 0.30 seconds)


In [10]:
#1h: Prepare drought data
clipshp = "ca-state-boundary/CA_STATE_TIGER2016.shp"
inputshp = "Drought_data/USDM_20201208.shp"
tempshp = "tempdrought.shp"
outputtif = "drought.tif"

#Clip the drought data to the California borders (it had ocean data as well)
#Convert to raster
arcpy.Clip_analysis(inputshp, clipshp, tempshp)
arcpy.MakeFeatureLayer_management(tempshp, "tempLayer")
arcpy.FeatureToRaster_conversion("tempLayer", "DM", outputtif, 0.1)
print(arcpy.GetMessages())

Start Time: Tuesday, December 15, 2020 7:47:47 PM
Succeeded at Tuesday, December 15, 2020 7:47:47 PM (Elapsed Time: 0.29 seconds)


In [23]:
#1i: Prepare burn area data (Rx and non-Rx)
#First I need to select only the fires from the last 20 years
#Then I need to convert to raster and set all the places with fires to value=1 and those without value=0
burninput = "California_Fire_Perimeters_1878_-_2019_shp/Burn_areas.shp"
rxburninput = "California_Fire_Perimeters_1878_-_2019_shp/Prescribed_burns.shp"

burnoutput = "burn.tif"
rxoutput = "rxburn.tif"

arcpy.MakeFeatureLayer_management(burninput, "burntemp")
arcpy.AddField_management("burntemp", "isburned", "LONG", 8)
arcpy.AddField_management("burntemp", "YEAR", "LONG", 8)
arcpy.CalculateField_management("burntemp", "YEAR", "0 if !YEAR_! == ' ' else !YEAR_!", "PYTHON3")
arcpy.CalculateField_management("burntemp", "isburned", "1 if !YEAR! > 1990 else 0", "PYTHON3")

arcpy.MakeFeatureLayer_management(rxburninput, "rxburntemp")
arcpy.AddField_management("rxburntemp", "isburned", "LONG", 8)
arcpy.AddField_management("rxburntemp", "YEAR", "LONG", 8)
arcpy.CalculateField_management("rxburntemp", "YEAR", "0 if !YEAR_! == ' ' else !YEAR_!", "PYTHON3")
arcpy.CalculateField_management("rxburntemp", "isburned", "1 if !YEAR! > 1990 else 0", "PYTHON3")


arcpy.FeatureToRaster_conversion("burntemp", "isburned", burnoutput)
arcpy.FeatureToRaster_conversion("rxburntemp", "isburned", rxoutput)

print(arcpy.GetMessages())

Start Time: Tuesday, December 15, 2020 12:14:39 PM
Succeeded at Tuesday, December 15, 2020 12:14:41 PM (Elapsed Time: 1.17 seconds)


In [18]:
#step 2:
#Use Raster calculator to normalize and apply importance weights
population  = "population.tif"
housing = "housing.tif"
risk = "risk.tif"
powerlines = "powerlines.tif"
wind = "wind.tif"
drought = "drought.tif"
pfrid = "pfrid.tif"
temperature = "temp.tif"
burnareas = "burn.tif"
rxburnareas = "rxburn.tif"

population_result = "populationrisk.tif"
housing_result = "housingrisk.tif"

pop_output = RasterCalculator([population, risk, powerlines, wind, drought, pfrid, temperature, burnareas, rxburnareas], ["pop", "risk", "power", "wind", "drt", "pfrid", "temp", "brn", "rxbrn"], "pop*(0.55*(risk/5)+(0.1*power)+(0.1*(wind/6))+(0.1*(drt/4))+(0.1*pfrid)+(0.05*((temp+1)/60))+(-0.05*brn)+(-0.05*rxbrn))", "UnionOf", "MeanOf")
house_output = RasterCalculator([housing, risk, powerlines, wind, drought, pfrid, temperature, burnareas, rxburnareas], ["house", "risk", "power", "wind", "drt", "pfrid", "temp", "brn", "rxbrn"], "house*(0.55*(risk/5)+(0.1*power)+(0.1*(wind/6))+(0.1*(drt/4))+(0.1*pfrid)+(0.05*((temp+1)/60))+(-0.05*brn)+(-0.05*rxbrn))", "UnionOf", "MeanOf")
pop_output.save(population_result)
house_output.save(housing_result)
print(arcpy.GetMessages())

Start Time: Tuesday, December 15, 2020 1:28:48 PM
Building Pyramids...
Succeeded at Tuesday, December 15, 2020 1:28:48 PM (Elapsed Time: 0.15 seconds)
